In [1]:
import numpy as np
import pandas as pd
import numpy.random as rnd
import scipy.linalg as la
from scipy.optimize import minimize
from qutip import *

# Linear inversion

In [2]:
import numpy as np

M1 = 0.5 * np.array([[2, -(1-1j), -(1+1j), 1],
                     [-(1+1j), 0, 1j, 0],
                     [-(1-1j), -1j, 0, 0],
                     [1, 0, 0, 0]]) #cite: 18

M2 = 0.5 * np.array([[0, -(1-1j), 0, 1],
                     [-(1+1j), 2, 1j, -(1+1j)],
                     [0, -1j, 0, 0],
                     [1, -(1+1j), 0, 0]]) #cite: 18

M3 = 0.5 * np.array([[0, 0, 0, 1],
                     [0, 0, 1j, -(1+1j)],
                     [0, -1j, 0, -(1-1j)],
                     [1, -(1-1j), -(1+1j), 2]]) #cite: 18

M4 = 0.5 * np.array([[0, 0, -(1+1j), 1],
                     [0, 0, 1j, 0],
                     [-(1-1j), -1j, 2, -(1-1j)],
                     [1, 0, -(1+1j), 0]]) #cite: 18

M5 = 0.5 * np.array([[0, 0, 2j, -(1+1j)],
                     [0, 0, (1-1j), 0],
                     [-2j, (1+1j), 0, 0],
                     [-(1-1j), 0, 0, 0]]) #cite: 18

M6 = 0.5 * np.array([[0, 0, 0, -(1+1j)],
                     [0, 0, (1-1j), 2j],
                     [0, (1+1j), 0, 0],
                     [-(1-1j), -2j, 0, 0]]) #cite: 19

M7 = 0.5 * np.array([[0, 0, 0, -(1+1j)],
                     [0, 0, -(1-1j), 2],
                     [0, -(1+1j), 0, 0],
                     [-(1-1j), 2, 0, 0]]) #cite: 19

M8 = 0.5 * np.array([[0, 0, 2, -(1+1j)],
                     [0, 0, -(1-1j), 0],
                     [2, -(1+1j), 0, 0],
                     [-(1-1j), 0, 0, 0]]) #cite: 2, 6, 7, 9, 10, 11, 12, 13, 14 (Note: This matrix definition appears to be spread across multiple lines in the source, so it has been reconstructed. The last row and column are empty according to the input.)

M9 = np.array([[0, 0, 0, 1j],
               [0, 0, -1j, 0],
               [0, 1j, 0, 0],
               [-1j, 0, 0, 0]]) #cite: 15

M10 = np.array([[0, 0, 0, 1],
                [0, 0, 1, 0],
                [0, 1, 0, 0],
                [1, 0, 0, 0]]) #cite: 15

M11 = np.array([[0, 0, 0, 1j],
                [0, 0, 1j, 0],
                [0, -1j, 0, 0],
                [-1j, 0, 0, 0]]) #cite: 16

M12 = 0.5 * np.array([[0, 2, 0, -(1+1j)],
                      [2, 0, -(1+1j), 0],
                      [0, -(1-1j), 0, 0],
                      [-(1-1j), 0, 0, 0]]) #cite: 18

M13 = 0.5 * np.array([[0, 0, 0, -(1+1j)],
                      [0, 0, -(1+1j), 0],
                      [0, -(1-1j), 0, 2],
                      [-(1-1j), 0, 2, 0]]) #cite: 18

M14 = 0.5 * np.array([[0, 0, 0, -(1-1j)],
                      [0, 0, -(1-1j), 0],
                      [0, -(1+1j), 0, -2j],
                      [-(1+1j), 0, 2j, 0]]) #cite: 18

M15 = 0.5 * np.array([[0, -2j, 0, -(1-1j)],
                      [2j, 0, (1-1j), 0],
                      [0, (1+1j), 0, 0],
                      [-(1+1j), 0, 0, 0]]) #cite: 18

M16 = np.array([[0, 0, 0, 1],
                [0, 0, -1, 0],
                [0, -1, 0, 0],
                [1, 0, 0, 0]]) #cite: 21

matrices = [M1, M2, M3, M4, M5, M6, M7, M8, M9, M10, M11, M12, M13, M14, M15, M16]
qutip_mtrx = [Qobj(M) for M in matrices]

M = dict(zip(range(1, 17), qutip_mtrx))
M1

array([[ 1. +0.j , -0.5+0.5j, -0.5-0.5j,  0.5+0.j ],
       [-0.5-0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ],
       [-0.5+0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ],
       [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ]])

In [3]:
# Phi-, Phi+, Decoherence contain coincidence counts
col_names = ['idx','A', 'B', 'HWPa', 'QWPa', 'HWPb', 'QWPb', 'Phi-', 'Phi+', 'Decoher.']
data = pd.read_excel('Tomography2025.xlsx', header=2, names=col_names, index_col='idx',usecols='A:J', nrows=16)
id_x, id_y = np.where(data.isna())
for id in zip(id_x, id_y):
    data.iloc[id[0], id[1]] = data.iloc[id[0], 6]

data

,A,B,HWPa,QWPa,HWPb,QWPb,Phi-,Phi+,Decoher.
idx,,,,,,,,,
1,H,H,36.0,84,22.0,38,3831,3672.0,18320.0
2,H,V,36.0,84,67.0,38,45,35.0,290.0
3,V,V,81.0,84,67.0,38,3655,3671.0,19310.0
4,V,H,81.0,84,22.0,38,71,75.0,430.0
5,R,H,81.0,129,22.0,38,2218,2218.0,2218.0
6,R,V,81.0,129,67.0,38,1607,1607.0,1607.0
7,D,V,58.5,129,67.0,38,1577,1577.0,1577.0
8,D,H,58.5,129,22.0,38,2219,2219.0,2219.0
9,D,R,58.5,129,22.0,-7,2305,2305.0,2305.0


In [4]:
psi0 = ['Phi-', 'Phi+', 'Decoher.']
densities = dict(zip(psi0, [Qobj(np.zeros((4,4)))]*3))
for state in psi0:
    counts = data[state]
    for i in range(1, 17):
        densities[state] += counts[i]*M[i]
    densities[state] /= np.sum(counts[0:4])

    print(f"Density matrix for {state}:")
    display(densities[state].tidyup())
    print("---------------------------------------------------------------------------------------------")
    print(f"Trace: {densities[state].tr()}")
    print(f"Eigenvalues: {densities[state].eigenenergies()}")
    print("---------------------------------------------------------------------------------------------")
    print("---------------------------------------------------------------------------------------------", '\n')

Density matrix for Phi-:


Quantum object: dims=[[4], [4]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.50394633+0.j         -0.00052618-0.02157327j  0.03525388+0.03512234j
  -0.36700868+0.12825572j]
 [-0.00052618+0.02157327j  0.00591949+0.j         -0.18468824+0.24467245j
  -0.0359116 -0.03196527j]
 [ 0.03525388-0.03512234j -0.18468824-0.24467245j  0.00933965+0.j
  -0.03617469+0.0486714j ]
 [-0.36700868-0.12825572j -0.0359116 +0.02604578j -0.03617469-0.0486714j
   0.48079453+0.j        ]]

---------------------------------------------------------------------------------------------
Trace: (1+0j)
Eigenvalues: [-0.30029286-1.33749051e-04j  0.10270105-5.38157689e-05j
  0.3028792 -1.71078345e-05j  0.8947126 +2.04672655e-04j]
---------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------- 

Density matrix for Phi+:


Quantum object: dims=[[4], [4]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.49268751+0.j          0.01080102-0.03334228j  0.04635717+0.046223j
   0.3286596 +0.1308198j ]
 [ 0.01080102+0.03334228j  0.0046961 +0.j         -0.13793103+0.23956796j
  -0.03703207-0.03300684j]
 [ 0.04635717-0.046223j   -0.13793103-0.23956796j  0.01006306+0.j
  -0.03823964+0.05098618j]
 [ 0.3286596 -0.1308198j  -0.03703207+0.02831075j -0.03823964-0.05098618j
   0.49255333+0.j        ]]

---------------------------------------------------------------------------------------------
Trace: (1+0j)
Eigenvalues: [-0.27134174-5.92273604e-05j  0.09084632-2.32284476e-04j
  0.33172329+1.32462652e-04j  0.84877213+1.59049184e-04j]
---------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------- 

Density matrix for Decoher.:


Quantum object: dims=[[4], [4]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.47770535+0.j         -0.19220339+0.18782269j -0.18659713-0.18662321j
   0.53196871+0.02542373j]
 [-0.19220339-0.18782269j  0.00756193+0.j         -0.05791395+0.44938722j
  -0.21441982-0.21363755j]
 [-0.18659713+0.18662321j -0.05791395-0.44938722j  0.01121252+0.j
  -0.21595828+0.21843546j]
 [ 0.53196871-0.02542373j -0.21441982+0.20607562j -0.21595828-0.21843546j
   0.50352021+0.j        ]]

---------------------------------------------------------------------------------------------
Trace: (1+0j)
Eigenvalues: [-0.4451866 -7.39410810e-05j -0.04726561-5.68068744e-04j
  0.11451926-5.35860403e-04j  1.37793296+1.17787023e-03j]
---------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------- 



# Maximum likelihood estimation

In [ ]:
HH = tensor(basis(2, 0), basis(2, 0))
HV = tensor(basis(2, 0), basis(2, 1))
VH = tensor(basis(2, 1), basis(2, 0))
VV = tensor(basis(2, 1), basis(2, 1))

def a(h, q):
    return 1/np.sqrt(2) * (np.sin(2*h) - 1j*np.sin(2*(h-q)))
def b(h, q):
    return -1/np.sqrt(2) * (np.cos(2*h) + 1j*np.cos(2*(h-q)))

projection_states = []
# A	B	HWPa	QWPa	HWPb	QWPb	Phi-	Phi+	Decoher.
for i in range(16):
    row = data.iloc[i, :]
    projection_states.append(Qobj(
        a(row['HWPa'], row['QWPa'])*a(row['HWPb'], row['QWPb'])*HH +
        a(row['HWPa'], row['QWPa'])*b(row['HWPb'], row['QWPb'])*HV +
        b(row['HWPa'], row['QWPa'])*a(row['HWPb'], row['QWPb'])*VH +
        b(row['HWPa'], row['QWPa'])*b(row['HWPb'], row['QWPb'])*VV
        ))

print("|psi_0>:")
display(projection_states[0])

def rho_p(t):
    T = Qobj(
        [[t[0], 0, 0, 0],
         [t[4]+1j*t[5], t[1], 0, 0],
         [t[10]+1j*t[11], t[6]+1j*t[7], t[2], 0],
         [t[14]+1j*t[15], t[12]+1j*t[13], t[8]+1j*t[9], t[3]]],
         dims=[[2,2],[2,2]]
    )
    G = T.dag()*T
    return  (G/G.tr()).tidyup()

c = [1]*16
rho = rho_p(c)
print(f'Density matrix:')
display(rho)
print(f'Trace: {rho.tr()}')
print(f'Eigenvalues: {rho.eigenenergies()}')

#print(f'<psi|rho|ket>/N = {projection_states[0].dag() * rho * projection_states[0]}')


def likelihood(t, data):
    out = 0
    measures = np.array(data)
    N = np.sum(measures[0:4])
    for i in range(16):
        psi = projection_states[i]
        bra_rho_ket = np.real(psi.dag() * rho_p(t) * psi)
        out += (N*bra_rho_ket - measures[i])**2 / (2*N*bra_rho_ket)
    return out
    
print(f'Likelihood: {likelihood([1]*16, data['Phi-'])}')

|psi_0>:


Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[-0.26894168+0.07868819j]
 [ 0.28337414-0.5975895j ]
 [-0.04118598+0.26828087j]
 [-0.40828987-0.49365261j]]

Density matrix:


Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.4375+0.j     0.3125-0.0625j 0.1875-0.0625j 0.0625-0.0625j]
 [0.3125+0.0625j 0.3125+0.j     0.1875-0.0625j 0.0625-0.0625j]
 [0.1875+0.0625j 0.1875+0.0625j 0.1875+0.j     0.0625-0.0625j]
 [0.0625+0.0625j 0.0625+0.0625j 0.0625+0.0625j 0.0625+0.j    ]]

Trace: 1.0
Eigenvalues: [0.00460126 0.03507357 0.11137304 0.84895213]
Likelihood: 20358.681362228934


In [6]:
state_name = 'Phi+'
density = densities[state_name]
rho0 = density.full()

def first_minor(matrix, row, col):
    return la.det(np.delete(np.delete(matrix, row, 0), col, 1))

def second_minor(matrix, rows, cols):
    return la.det(np.delete(np.delete(matrix, rows, 0), cols, 1))

Delta  = np.sqrt(la.det(rho0))
rho44  = np.sqrt(rho0[3, 3])
M11    = np.sqrt(first_minor(rho0, 0, 0))
M12    = first_minor(rho0, 0, 1)
M11_22 = np.sqrt(second_minor(rho0, [0,1], [0,1]))
M12_23 = second_minor(rho0, [0,1], [1,2])
M11_23 = second_minor(rho0, [0,1], [0,2])

t0 = [
    np.real(Delta/M11),
    np.real(M11/M11_22),
    np.real(M11_22/rho44),
    np.real(rho44),                 # t4
    np.real(M12/(M11*M11_22)),
    np.imag(M12/(M11*M11_22)),
    np.real(M11_23/(rho44*M11_22)),
    np.imag(M11_23/(rho44*M11_22)), # t8
    np.real(rho0[3,2]/rho44),
    np.imag(rho0[3,2]/rho44),
    np.real(M12_23/(rho44*M11_22)),
    np.imag(M12_23/(rho44*M11_22)), # t12
    np.real(rho0[3,1]/rho44),
    np.imag(rho0[3,1]/rho44),
    np.real(rho0[3,0]/rho44),
    np.imag(rho0[3,0]/rho44)
]

# print(f'Determinant: {Delta}')
# print(f'Minor^11: {M11}')
# print(f'Minor^11_22: {M11_22}')

res = minimize(lambda t: likelihood(t, data[state_name]), t0)
sol = res.x
print(f'Best solution with likelihood {likelihood(sol, data[state_name])}:')
display(sol)
rho_ = rho_p(sol)
print(f'Eigenvalues: {rho_.eigenenergies()}')
print(f'Tr[⍴] = {(rho_).tr()}')
print(f'Tr[⍴^2] = {(rho_*rho_).tr()}')
rho_.tidyup()

Best solution with likelihood 1158.3072379968423:


array([-4.65502727e-05, -1.59203432e-05, -2.33564086e+01,  1.74998304e+01,
        6.18973046e-05, -5.69997887e-05,  4.14250464e+00, -1.37148209e+00,
       -2.35606675e+00, -1.28993188e+01,  6.06459001e+00, -6.08820977e+00,
        1.29451736e+00, -2.03282394e+01,  1.66174446e+01,  9.11661955e+00])

Eigenvalues: [1.27754569e-14 3.13857368e-12 2.34015786e-01 7.65984214e-01]
Tr[⍴] = 1.0
Tr[⍴^2] = 0.6414952044025197


Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.22905996+0.j         -0.06893561-0.17596193j -0.15781844-0.17721593j
   0.15380177-0.08437833j]
 [-0.06893561+0.17596193j  0.22951303+0.j          0.08590001-0.05110426j
   0.01198133+0.18814681j]
 [-0.15781844+0.17721593j  0.08590001+0.05110426j  0.37945838+0.j
  -0.02180644+0.11938887j]
 [ 0.15380177+0.08437833j  0.01198133-0.18814681j -0.02180644-0.11938887j
   0.16196863+0.j        ]]

In [7]:
best = 1e10
sol  = 0
for _ in range(20):
    res = minimize(lambda t: likelihood(t, data[state_name]), rnd.normal(0, 10, 16))
    likelihood_ = likelihood(res.x, data[state_name])
    if best > likelihood_:
        best = likelihood_
        sol = res.x
best, sol

(1158.3072379895746,
 array([ 6.78400581e-06,  3.87781651e-06, -1.84004695e+01,  1.37865880e+01,
         5.91668167e-07, -5.23653559e-06,  3.26353219e+00, -1.08048503e+00,
        -1.85614310e+00, -1.01622644e+01,  4.77772488e+00, -4.79634661e+00,
         1.01981836e+00, -1.60148418e+01,  1.30914501e+01,  7.18217759e+00]))

# Fidelity, Von Neuman entropy, Concurrence

In [8]:
def computeRelevantQuantities(rho, psi0 = None):

    eigenvalues = rho.eigenenergies()
    if np.any(np.imag(eigenvalues) > 1e-8):
        print("Eigenvalues are complex!")
    else :
        eigenvalues = np.real(eigenvalues)
    entropy = -np.sum(eigenvalues*np.log(eigenvalues))   

    #concurr = concurrence(rho)
    concurr = 10
    if psi0 != None:
        fidelity = psi0.dag() * rho * psi0
        return fidelity, entropy, concurr
    else:
        return entropy, concurr

In [9]:
computeRelevantQuantities(rho_, bell_state('10'))

((0.3903857138915376+6.938893903907228e-18j), 0.5440833151231359, 10)

# Errors

In [10]:
def generateEnsamble(measures, N = 100):
    '''
    Generates N random density matrices and computes parameters errors as standard deviations of the parameters population
    '''
    lambda_par = measures

    simulated_data = rnd.poisson(lambda_par, (N, lambda_par.shape[0]))
    entropies, concurrences = np.zeros((N)), np.zeros((N))
    for i in range(N):
        solution = minimize(lambda t: likelihood(t, simulated_data[i, :]), t0)
        simulated_rho = rho_p(solution.x)
        simulated_entropy, simulated_concurr = computeRelevantQuantities(simulated_rho)
        entropies[i] = simulated_entropy
        concurrences[i] = simulated_concurr
    return entropies, concurrences


entropies, concurrences = generateEnsamble(data['Phi-'])
print(f"Mean entropy = {np.mean(entropies)}")
print(f"STD entropy = {np.std(entropies)}")
print(f"Mean concurrence = {np.mean(concurrences)}")
print(f"STD concurrence = {np.mean(concurrences)}")


/var/folders/bq/c0v_5_v15yv4wrwpv0d4mc1r0000gn/T/ipykernel_96094/2441744483.py:8: RuntimeWarning: invalid value encountered in log
  entropy = -np.sum(eigenvalues*np.log(eigenvalues))


Mean entropy = nan
STD entropy = nan
Mean concurrence = 10.0
STD concurrence = 10.0
